## **Testing Audio Devices**

In [1]:
import pyaudio

def list_audio_devices():
    """
    List all available audio devices using PyAudio.

    This function initializes PyAudio, retrieves the list of audio devices,
    and prints their names. It also includes error handling to ensure proper
    cleanup of resources.
    """
    try:
        p = pyaudio.PyAudio()
        print("Available audio devices:")
        for ii in range(p.get_device_count()):
            device_name = p.get_device_info_by_index(ii).get('name')
            print(f"{ii}: {device_name}")
    except Exception as e:
        print(f"An error occurred while listing audio devices: {e}")
    finally:
        # Ensure PyAudio resources are released
        if 'p' in locals():
            p.terminate()

# Call the function to list audio devices
list_audio_devices()

Available audio devices:
0: Microsoft Sound Mapper - Input
1: Surface Stereo Microphones (Sur
2: Headset (Pablo’s AirPods #4)
3: Microphone (Lumina Camera - Raw
4: Echo Cancelling Speakerphone (M
5: Microsoft Sound Mapper - Output
6: Headphones (Pablo’s AirPods #4)
7: Speakers (Dell USB Audio)
8: Surface Omnisonic Speakers (Sur
9: Echo Cancelling Speakerphone (M
10: Primary Sound Capture Driver
11: Surface Stereo Microphones (Surface High Definition Audio)
12: Headset (Pablo’s AirPods #4)
13: Microphone (Lumina Camera - Raw)
14: Echo Cancelling Speakerphone (Microsoft Audio Dock)
15: Primary Sound Driver
16: Headphones (Pablo’s AirPods #4)
17: Speakers (Dell USB Audio)
18: Surface Omnisonic Speakers (Surface High Definition Audio)
19: Echo Cancelling Speakerphone (Microsoft Audio Dock)
20: Headphones (Pablo’s AirPods #4)
21: Speakers (Dell USB Audio)
22: Surface Omnisonic Speakers (Surface High Definition Audio)
23: Echo Cancelling Speakerphone (Microsoft Audio Dock)
24: Surface Stereo

In [2]:
import pyaudio
import wave

def test_microphone():
    """
    Test the microphone by recording audio and playing it back.

    This function captures audio from the default input device (microphone),
    saves it to a temporary WAV file, and plays it back to ensure the microphone
    is working correctly.
    """
    # Audio configuration
    chunk = 1024  # Number of frames per buffer
    format = pyaudio.paInt16  # 16-bit audio format
    channels = 1  # Mono audio
    rate = 44100  # Sampling rate (44.1 kHz)
    record_seconds = 5  # Duration of the recording
    output_filename = "test_audio.wav"

    # Initialize PyAudio
    p = pyaudio.PyAudio()

    try:
        # Open the microphone stream
        print("Recording...")
        stream = p.open(format=format,
                        channels=channels,
                        rate=rate,
                        input=True,
                        frames_per_buffer=chunk)

        frames = []

        # Record audio in chunks
        for _ in range(0, int(rate / chunk * record_seconds)):
            data = stream.read(chunk)
            frames.append(data)

        print("Recording complete. Saving audio...")

        # Save the recorded audio to a WAV file
        with wave.open(output_filename, 'wb') as wf:
            wf.setnchannels(channels)
            wf.setsampwidth(p.get_sample_size(format))
            wf.setframerate(rate)
            wf.writeframes(b''.join(frames))

        print(f"Audio saved to {output_filename}. Playing back...")

        # Play back the recorded audio
        stream.stop_stream()
        stream.close()

        # Open the WAV file for playback
        wf = wave.open(output_filename, 'rb')
        playback_stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                                 channels=wf.getnchannels(),
                                 rate=wf.getframerate(),
                                 output=True)

        # Read and play audio data
        data = wf.readframes(chunk)
        while data:
            playback_stream.write(data)
            data = wf.readframes(chunk)

        playback_stream.stop_stream()
        playback_stream.close()

        print("Playback complete.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Terminate PyAudio
        p.terminate()

# Run the microphone test
test_microphone()

Recording...
Recording complete. Saving audio...
Audio saved to test_audio.wav. Playing back...
Playback complete.


## **Define Clients**

In [1]:
import logging 
import os
# set the directory to the location of the script
try:
    target_directory = os.getenv("TARGET_DIRECTORY", os.getcwd())  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

In [2]:
from src.speech.speech_recognizer import SpeechRecognizer, StreamingSpeechRecognizer
from src.speech.text_to_speech import SpeechSynthesizer
from openai import AzureOpenAI

# Ensure clients are initialized only if not already defined
if 'az_speech_recognizer_client' not in locals():
    az_speech_recognizer_client = SpeechRecognizer()

if 'az_speech_recognizer_stream_client' not in locals():
    az_speech_recognizer_stream_client = StreamingSpeechRecognizer(vad_silence_timeout_ms=5000)

if 'az_speach_synthesizer_client' not in locals():
    az_speach_synthesizer_client = SpeechSynthesizer()

# Ensure Azure OpenAI client is initialized only if not already defined
if 'client' not in locals():
    client = AzureOpenAI(
        api_version="2025-02-01-preview",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    )

Using voice: en-US-JennyMultilingualNeural


## **Azure AI Speech**

In [ ]:
# Initialize
recognizer = StreamingSpeechRecognizer(
    vad_silence_timeout_ms=3000
)

# Buffers
all_text_live = ""
final_transcripts = []

# Internal tracker to prevent double-processing
last_final_text = None

# Callbacks
def on_partial(text: str):
    global all_text_live
    print(f"Partial: {text}")
    all_text_live = text

def on_final(text: str):
    global all_text_live, last_final_text

    # Skip duplicate finalizations
    if text == last_final_text:
        return

    last_final_text = text

    print(f"Final: {text}")
    final_transcripts.append(text)
    all_text_live = ""  # Clear the live buffer

# Attach
recognizer.set_partial_result_callback(on_partial)
recognizer.set_final_result_callback(on_final)

# Start recognizing
recognizer.start()


2025-04-15 20:03:43,343 - micro - MainProcess - INFO     Starting continuous speech recognition with VAD using asynchronous call... (speech_recognizer.py:start:134)
INFO:micro:Starting continuous speech recognition with VAD using asynchronous call...
2025-04-15 20:03:44,056 - micro - MainProcess - INFO     Continuous speech recognition started. (speech_recognizer.py:start:170)
INFO:micro:Continuous speech recognition started.


Partial: hello how
Partial: hello how are you
Partial: hello how are you doing
Partial: hello how are you doing are you doing well
Final: Hello, how are you doing? Are you doing well?


In [ ]:
full_conversation = " ".join(final_transcripts) + " " + all_text_live
full_conversation

'Hello, how are you doing? Are you doing well? '

Partial: hey babu
Partial: hey babu anybody
Partial: hey babu anybody how are you
Final: Hey Babu, anybody, How are you?
Partial: too often
Partial: lately
Partial: lately i
Partial: too often lately i mean man
Partial: too often lately i mean man lately i
Partial: too often lately i mean man lately i've been
Partial: too often lately i mean man lately i've been but
Partial: too often lately i mean man lately i've been but this is
Partial: too often lately i mean man lately i've been but this is this is
Partial: too often lately i mean man lately i've been but this is this is kind
Partial: too often lately i mean man lately i've been but this is this is kind of
Partial: too often lately i mean man lately i've been but this is this is kind of fun
Partial: too often lately i mean man lately i've been but this is this is kind of fun i see it
Partial: too often lately i mean man lately i've been but this is this is kind of fun i see it we
Partial: too often lately i mean man lately i've be

In [7]:
# Start recognizing
recognizer.stop()

2025-04-15 19:48:03,865 - micro - MainProcess - INFO     Stopping continuous speech recognition... (speech_recognizer.py:stop:186)
INFO:micro:Stopping continuous speech recognition...
2025-04-15 19:48:03,927 - micro - MainProcess - INFO     Session stopped: SessionEventArgs(session_id=79dc5894d1e64d8e97bf9cfcc776ea0a) (speech_recognizer.py:_on_session_stopped:216)
INFO:micro:Session stopped: SessionEventArgs(session_id=79dc5894d1e64d8e97bf9cfcc776ea0a)


In [3]:
import time
SILENCE_THRESHOLD = 5  # seconds of silence to stop recognition

def handle_speech_recognition() -> str:
    global all_text_live, final_transcripts, last_final_text

    print("Starting microphone recognition...")
    final_transcripts.clear()
    all_text_live = ""
    last_final_text = None

    def on_partial(text: str) -> None:
        global all_text_live
        all_text_live = text
        print(f"Partial recognized: {text}")
        recognizer.stop_speaking()

    def on_final(text: str) -> None:
        global all_text_live, final_transcripts, last_final_text
        if text and text != last_final_text:
            final_transcripts.append(text)
            last_final_text = text
            all_text_live = ""
            print(f"Finalized text: {text}")

    az_speech_recognizer_client.set_partial_result_callback(on_partial)
    az_speech_recognizer_client.set_final_result_callback(on_final)

    az_speech_recognizer_client.start()
    print("🎤 Listening... (speak now)")

    start_time = time.time()
    while not final_transcripts and (time.time() - start_time < SILENCE_THRESHOLD):
        time.sleep(0.05)

    az_speech_recognizer_client.stop()
    print("🛑 Recognition stopped.")

    return " ".join(final_transcripts) + " " + all_text_live

# Example usage
recognized_text = handle_speech_recognition()
print(f"Recognized text: {recognized_text}")

Starting microphone recognition...


NameError: name 'final_transcripts' is not defined

## **Streaming Text with gpt4o-mini**

In [9]:
response = client.chat.completions.create(
    stream=True,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": f'{full_conversation}',
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
)
full_response = ""
for update in response:
    if update.choices:
        chunk = update.choices[0].delta.content or ""
        full_response += chunk
        print(chunk, end="", flush=True)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?

## **Speach to Text - Return the Audio to the User**

In [10]:
import time
az_speach_synthesizer_client.start_speaking_text(
                    text=full_response,
                )
time.sleep(4)
az_speach_synthesizer_client.stop_speaking()

2025-04-15 15:36:26,113 - micro - MainProcess - INFO     [🔊] Starting streaming speech synthesis for text: Hello! I'm just a computer pro... (text_to_speech.py:start_speaking_text:36)
INFO:micro:[🔊] Starting streaming speech synthesis for text: Hello! I'm just a computer pro...
2025-04-15 15:36:30,122 - micro - MainProcess - INFO     [🛑] Stopping speech synthesis... (text_to_speech.py:stop_speaking:43)
INFO:micro:[🛑] Stopping speech synthesis...


## **Streaming Audio**

In [11]:
tts_sentence_end = [ ".", "!", "?", ";", "。", "！", "？", "；", "\n" ]
completion = client.chat.completions.create(
    stream=True,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": f'{full_conversation}',
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
)

collected_messages = []
last_tts_request = None

for chunk in completion:
    if len(chunk.choices) > 0:
        chunk_text = chunk.choices[0].delta.content
        if chunk_text:
            collected_messages.append(chunk_text)
            if chunk_text in tts_sentence_end:
                text = "".join(collected_messages).strip()
                last_tts_request = az_speach_synthesizer_client.start_speaking_text(text)
                collected_messages.clear()

2025-04-15 15:36:37,429 - micro - MainProcess - INFO     [🔊] Starting streaming speech synthesis for text: Hello!... (text_to_speech.py:start_speaking_text:36)
INFO:micro:[🔊] Starting streaming speech synthesis for text: Hello!...
2025-04-15 15:36:37,485 - micro - MainProcess - INFO     [🔊] Starting streaming speech synthesis for text: I'm just a computer program, s... (text_to_speech.py:start_speaking_text:36)
INFO:micro:[🔊] Starting streaming speech synthesis for text: I'm just a computer program, s...
2025-04-15 15:36:37,491 - micro - MainProcess - INFO     [🔊] Starting streaming speech synthesis for text: How can I assist you today?... (text_to_speech.py:start_speaking_text:36)
INFO:micro:[🔊] Starting streaming speech synthesis for text: How can I assist you today?...


## **Adding Prompt**

In [3]:
from typing import List, Dict

from app.backend.prompt_manager import PromptManager

prompt_manager = PromptManager()
systemp_prompt = prompt_manager.get_prompt("voice_agent_system.jinja")

Templates found: ['voice_agent_system.jinja', 'voice_agent_user.jinja']


## **Adding Tools**

In [4]:
from app.backend.tools import available_tools
from app.backend.functions import (
    schedule_appointment,
    refill_prescription,
    lookup_medication_info,
    evaluate_prior_authorization,
    escalate_emergency,
    authenticate_user
)

In [5]:
conversation_history: List[Dict[str, str]] = [
    {"role": "system", "content": systemp_prompt},
    {
        "role": "user",
        "content": "Hello, my name is Pablo Salvador. "
        "this is an emergency. I need to schedule an appointment with my doctor ASAP.",
    },
]

In [6]:
# Mapping tool names to actual Python async functions
function_mapping = {
    "schedule_appointment": schedule_appointment,
    "refill_prescription": refill_prescription,
    "lookup_medication_info": lookup_medication_info,
    "evaluate_prior_authorization": evaluate_prior_authorization,
    "escalate_emergency": escalate_emergency,
    "authenticate_user": authenticate_user,
}


In [40]:
import json

response = client.chat.completions.create(
        messages=conversation_history,
        tools=available_tools,
        tool_choice="auto",
        max_tokens=4096,
        temperature=0.5,
        top_p=1.0,
        model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
    )

# Process the model's response
response_message = response.choices[0].message
conversation_history.append(response_message)

print("Model's response:")  
print(response_message)  

if response_message.tool_calls:
    for tool_call in response_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        # Check if the function name is in the mapping
        if function_name in function_mapping:
            # Call the corresponding Python async function
            result = await function_mapping[function_name](**function_args)
            print(f"Function result: {result}")  
            conversation_history.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": result,
            })
else:
    print("No tool calls were made by the model.")

# Second API call: Get the final response from the model
final_response = client.chat.completions.create(
    messages=conversation_history,
    max_tokens=4096,
    temperature=0.5,
    top_p=1.0,
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
)   

print("Final response:")
print(final_response.choices[0].message.content)

Model's response:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9gnGBAJNhJ0EwW6KEx5xK81K', function=Function(arguments='{"reason":"urgent appointment needed"}', name='escalate_emergency'), type='function')])
Function result: 🚨 Emergency escalation triggered: urgent appointment needed. A human healthcare agent is now being connected.
Final response:
Hello Pablo, I understand this is an emergency, and I've escalated your request to a human healthcare agent who will assist you right away. Please hold on for a moment while they connect with you. Your health and safety are our top priority.


## **Making Tools work in Streaming Fashion**

In [23]:
conversation_history: List[Dict[str, str]] = [
    {"role": "system", "content": systemp_prompt},
    {
        "role": "user",
        "content": "Hello, my name is Pablo Salvador. "
        "this is an emergency. I need to schedule an appointment with my doctor ASAP.",
    },
]


In [24]:
# initial user message    
tool_call_accumulator = ""

# First API call: Ask the model to use the function
response = client.chat.completions.create(
    model= os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
    messages=conversation_history,
    tools=available_tools,
    tool_choice="auto",
    stream=True  # this time, we set stream=True
)

# process the model 
for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        print(f"delta: {delta}") # print delta from the chunk for learning
        
        if delta.tool_calls:
                if(delta.tool_calls[0].function.name):
                    tool_name = chunk.choices[0].delta.tool_calls[0].function.name
                    tool_id = chunk.choices[0].delta.tool_calls[0].id
                    conversation_history.append(delta)
                
                if(chunk.choices[0].delta.tool_calls[0].function.arguments):    
                    tool_call_accumulator+=delta.tool_calls[0].function.arguments

# print function related outputs                    
print(f"tool_name:{tool_name}")   
print(f"tool_id:{tool_id}")    
print(f"tool_call_accumulator:{tool_call_accumulator}")   
# Check if the function name is in the mapping
if tool_name in function_mapping:
    # Call the corresponding Python async function
    result = await function_mapping[function_name](**function_args)
    print(f"Function result: {result}")  
    conversation_history.append({
        "tool_call_id": tool_id,
        "role": "tool",
        "name": tool_name,
        "content": result,
    })

#print(f"messages: {messages}") 
print("=============================================")  
    
#  second API call: Get the final response with stream
response = client.chat.completions.create(
    model= os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
    messages=conversation_history,
    temperature=0.7,
    stream=True  # this time, we set stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        #print(f"delta: {delta}")
        if delta.content:
            print(delta.content, end="", flush=True)

delta: ChoiceDelta(content=None, function_call=None, refusal=None, role='assistant', tool_calls=[ChoiceDeltaToolCall(index=0, id='call_xkkR6tXUIgsxnHIz7RwaCN28', function=ChoiceDeltaToolCallFunction(arguments='', name='escalate_emergency'), type='function')])
delta: ChoiceDelta(content=None, function_call=None, refusal=None, role=None, tool_calls=[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='{"', name=None), type=None)])
delta: ChoiceDelta(content=None, function_call=None, refusal=None, role=None, tool_calls=[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='reason', name=None), type=None)])
delta: ChoiceDelta(content=None, function_call=None, refusal=None, role=None, tool_calls=[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='":"', name=None), type=None)])
delta: ChoiceDelta(content=None, function_call=None, refusal=None, role=None, tool_calls=[ChoiceDeltaToolCall(index=0

NameError: name 'function_name' is not defined

## **Run the Conversation**

In [25]:
conversation_history: List[Dict[str, str]] = [
    {"role": "system", "content": systemp_prompt},
    {
        "role": "user",
        "content": "Hello, my name is Pablo Salvador. "
        "this is an emergency. I need to schedule an appointment with my doctor ASAP.",
    },
]

In [26]:
conversation_history: List[Dict[str, str]] = [
    {"role": "system", "content": systemp_prompt},
    {
        "role": "user",
        "content": "Hello, my name is Pablo Salvador. "
    },
]

In [27]:
import json
import os
from typing import Any, Dict, List

tts_sentence_end = [".", "!", "?", ";", "。", "！", "？", "；", "\n"]

async def handle_chat(conversation_history: List[Dict[str, Any]]) -> None:
    """
    Handles a full streaming chat round with Azure OpenAI GPT-4o,
    correctly executes tool calls, synthesizes responses, and continues reasoning.
    """
    tool_name = None
    function_call_arguments = ""
    tool_call_id = None
    last_tts_request = None
    collected_messages: List[str] = []

    # 🔁 FIRST STREAMING RESPONSE (may include tool call)
    response = client.chat.completions.create(
        stream=True,
        messages=conversation_history,
        tools=available_tools,
        tool_choice="auto",
        max_tokens=4096,
        temperature=0.5,
        top_p=1.0,
        model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
    )

    for chunk in response:
        if chunk.choices:
            delta = chunk.choices[0].delta

            if delta.tool_calls:
                if(delta.tool_calls[0].function.name):
                    tool_name = chunk.choices[0].delta.tool_calls[0].function.name
                    tool_id = chunk.choices[0].delta.tool_calls[0].id
                    conversation_history.append(delta)
                
                if(chunk.choices[0].delta.tool_calls[0].function.arguments):    
                    function_call_arguments+=delta.tool_calls[0].function.arguments
            
            elif delta.content:
                chunk_text = chunk.choices[0].delta.content
                if chunk_text:
                    collected_messages.append(chunk_text)
                    if chunk_text in tts_sentence_end:
                        text = "".join(collected_messages).strip()
                        last_tts_request = az_speach_synthesizer_client.start_speaking_text(text)
                        collected_messages.clear()

    # 🧠 If tool call was detected, execute it
    if tool_name:
        print(f"tool_name:{tool_name}")   
        print(f"tool_id:{tool_id}")    
        print(f"function_call_arguments:{function_call_arguments}")  
        try:
            parsed_args = json.loads(function_call_arguments.strip())
            function_to_call = function_mapping.get(tool_name)

            if function_to_call:
                result = await function_to_call(parsed_args)

                print(f"✅ Function `{tool_name}` executed. Result: {result}")

                conversation_history.append({
                    "tool_call_id": tool_id,
                    "role": "tool",
                    "name": tool_name,
                    "content": result,
                })

                # 🧠 SECOND STREAMING CALL AFTER TOOL EXECUTION
                second_response = client.chat.completions.create(
                    stream=True,
                    messages=conversation_history,
                    temperature=0.5,
                    top_p=1.0,
                    max_tokens=4096,
                    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
                )

                collected_messages = []

                for chunk in second_response:
                    if chunk.choices:
                        delta = chunk.choices[0].delta
                        if hasattr(delta, "content") and delta.content:
                            chunk_message = delta.content
                            collected_messages.append(chunk_message)
                            if chunk_message.strip() in tts_sentence_end:
                                text = ''.join(collected_messages).strip()
                                if text:
                                    az_speach_synthesizer_client.start_speaking_text(text)
                                    collected_messages.clear()

                final_text = ''.join(collected_messages).strip()
                if final_text:
                    conversation_history.append({"role": "assistant", "content": final_text})

        except json.JSONDecodeError as e:
            print(f"❌ Error parsing function arguments: {e}")

    else:
        # Append the assistant message if no function call was made
        final_text = ''.join(collected_messages).strip()
        if final_text:
            conversation_history.append({"role": "assistant", "content": final_text})
            print(f"✅ Final assistant message: {final_text}")


In [22]:
await handle_chat(conversation_history)

tool_name:escalate_emergency
tool_id:call_hbMdkWeNNrbjDUEhUih6vQEK
function_call_arguments:{"reason":"urgent appointment needed"}
✅ Function `escalate_emergency` executed. Result: 🚨 Emergency escalation triggered: {'reason': 'urgent appointment needed'}. A human healthcare agent is now being connected.


2025-04-15 00:42:22,451 - micro - MainProcess - INFO     [🔊] Starting streaming speech synthesis for text: I understand that this is an e... (text_to_speech.py:start_speaking_text:44)
INFO:micro:[🔊] Starting streaming speech synthesis for text: I understand that this is an e...
2025-04-15 00:42:22,529 - micro - MainProcess - INFO     [🔊] Starting streaming speech synthesis for text: I've escalated your request, a... (text_to_speech.py:start_speaking_text:44)
INFO:micro:[🔊] Starting streaming speech synthesis for text: I've escalated your request, a...
2025-04-15 00:42:22,535 - micro - MainProcess - INFO     [🔊] Starting streaming speech synthesis for text: Please hold on for a moment.... (text_to_speech.py:start_speaking_text:44)
INFO:micro:[🔊] Starting streaming speech synthesis for text: Please hold on for a moment....


In [65]:
az_speach_synthesizer_client.stop_speaking()

2025-04-14 17:05:05,507 - micro - MainProcess - INFO     [🛑] Stopping speech synthesis... (text_to_speech.py:stop_speaking:55)
INFO:micro:[🛑] Stopping speech synthesis...
